# DATA CHALLENGE :   HOUSE  PRICE FORCASTING

## 1. DATA PREPROCESSING :

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
filepath='C:\\Users\\Anass\\Desktop\\Datachallenge elements'
X_train=pd.read_csv('X_train_J01Z4CN.csv') 
Y_train=pd.read_csv('y_train_OXxrJt1.csv')
Y_test=pd.read_csv('y_random_MhJDhKK.csv')
X_test=pd.read_csv('X_test_BEhvxAN.csv')


In [ ]:
X_train.head()
Y_train.head()